In [ ]:
import os
# import httpx

from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter

from openinference.instrumentation.pydantic_ai import OpenInferenceSpanProcessor

from openai import AsyncOpenAI
from pydantic import BaseModel, Field
from pydantic_ai import Agent, ModelRetry, RunContext
from pydantic_ai.providers.openai import OpenAIProvider
from pydantic_ai.models.openai import OpenAIChatModel

from pydantic_ai.models.instrumented import InstrumentationSettings

MLFLOW_ENDPOINT = "http://localhost:9041/v1/traces"
MLFLOW_EXPERIMENT_ID = "0" # Default 

# OTel Provider 설정
provider = TracerProvider()
provider.add_span_processor(OpenInferenceSpanProcessor())
provider.add_span_processor(
    BatchSpanProcessor(
        OTLPSpanExporter(
            endpoint=MLFLOW_ENDPOINT,
            headers={"x-mlflow-experiment-id": MLFLOW_EXPERIMENT_ID},
        )
    )
)
trace.set_tracer_provider(provider)

# Pydantic AI 계측
instrumentation = InstrumentationSettings(version=2)

In [2]:
# Setup Model
MODEL="Qwen3-VL-30B-A3B-Instruct"

# Passing AsyncOpenAI currently causes following error
# 'ASyncHttpxClientWrapper' object has no attribute '_state'
# llm_client=AsyncOpenAI(
#     base_url="http://localhost:9001/v1",
#     api_key="",
# )

llm_provider = OpenAIProvider(
    # openai_client=llm_client,
    base_url="http://localhost:9001/v1",
    api_key="",
)
model = OpenAIChatModel(MODEL, provider=llm_provider)

In [ ]:
# Simple Example
class WeatherInfo(BaseModel):
    location: str
    temperature: float = Field(description="Temperature in Celsius")
    condition: str
    humidity: int = Field(description="Humidity percentage")

# Create an agent with system prompts and tools
weather_agent = Agent(
    model=model,
    output_type=WeatherInfo,
    system_prompt="You are a helpful weather assistant. Always provide accurate weather information.",
    instrument=instrumentation
)

@weather_agent.tool
async def get_weather_data(ctx: RunContext[None], location: str) -> str:
    """Get current weather data for a location."""
    # This is a placeholder - use a real weather API
    mock_data = {
        "temperature": 22.5,
        "condition": "partly cloudy",
        "humidity": 65
    }
    return f"Weather in {location}: {mock_data}"

# Run the agent with tool usage

In [4]:
result = await weather_agent.run("What's the weather like in Paris??")
print(result)

AgentRunResult(output=WeatherInfo(location='Paris', temperature=22.5, condition='partly cloudy', humidity=65))
